In [2]:
import sys
import datasets
from dotenv import dotenv_values
from pathlib import Path

config = dotenv_values("./../../config/.env") # take environment variables from .env.
base_path = Path(config["BASE_PATH"])
sys.path.append(str(base_path/"code"))
from ZeDiAc.Annotators import ActiveAnnotator

iteration = 2

data = datasets.load_from_disk(base_path/f"data/processed/active-learning-iteration-{iteration}")

FileNotFoundError: Directory /home/andreas/Polybox/Project-Support-Material/Thesis/data/processed/active-learning-iteration-2 not found

In [2]:
import itertools
import pandas as pd
from dotenv import dotenv_values
from pathlib import Path

config = dotenv_values("./../../config/.env") # take environment variables from .env.
base_path = Path(config["BASE_PATH"])


In [3]:
annotations = pd.read_pickle(base_path/"data/labeling/Eval-LLMs.pkl")

In [5]:
annotations.shape[0]

2766

In [14]:
annotations.report_id.unique().shape[0]

7

In [16]:
annotations

,report_id,paragraph_nr,text,n_words,labeled,cohere_loss,cohere_unexpected,openai_loss,openai_unexpected
345824,3585,513,The corporate headquarters of the Company is l...,42,True,False,False,NaN,NaN
345825,3585,530,The Company reported net income of $.3 million...,108,True,True,False,NaN,NaN
345826,3585,531,Contributing factors towards the Company’s 200...,176,True,True,False,NaN,NaN
345827,3585,534,Gross premiums written for the year ended Dece...,238,True,True,False,NaN,NaN
345828,3585,535,Premiums written refers to the total amount of...,139,True,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1150793,BeazleyPLC-AR_2017,2301,Sidecar special purpose syndicate Specialty re...,67,False,None,None,NaN,NaN
1150794,BeazleyPLC-AR_2017,2305,Total shareholder return (TSR) The increase in...,25,False,None,None,NaN,NaN
1150795,BeazleyPLC-AR_2017,2307,Unearned premiums reserve The portion of premi...,25,False,None,None,NaN,NaN
1150796,BeazleyPLC-AR_2017,2309,If you have finished reading this report and n...,31,False,None,None,NaN,NaN


In [4]:
annotations.groupby(["report_id"]).count()

,paragraph_nr,text,n_words,labeled,cohere_loss,cohere_unexpected,openai_loss,openai_unexpected
report_id,,,,,,,,
3585,39,39,39,39,39,39,0,0
3935,1,1,1,1,1,1,0,0
4123,157,157,157,157,157,157,0,0
5110,382,382,382,382,382,382,0,0
5407,184,184,184,184,184,184,0,0
BaloiseHoldingLtd-AR_2015,934,934,934,934,934,934,0,0
BeazleyPLC-AR_2017,1069,1069,1069,1069,886,886,0,0


In [6]:
annotations.labeled.sum()

2583

In [13]:
print(annotations.cohere_loss.apply(lambda x: x == "True").sum())
annotations.cohere_unexpected.apply(lambda x: x == "True").sum()

261


151

In [7]:

df = pd.read_pickle(base_path/"data/labeling/active-learning-iteration-2.pkl")
df_l = df[df.labeled]
df_l.strategy.value_counts()
df_l["sampling-strategy"] = df_l.strategy.apply(lambda x: "least confidence" if x.split("_")[0] == "lc" else "$\mathcal{F}_{+}$")
df_l["class"] = df_l.strategy.apply(lambda x: x.split("_")[1] if x.split("_")[0] == "lc" else x.split("_")[2])
df_l["iteration"] = df_l.strategy.apply(lambda x: int(x.split("_")[-1])+1 if len(x.split("_")) > 3 else 1)

table_path = base_path/"writing"/"MSc-Thesis-Emerging-Risks"/"tables"/"active-learning-stats"
groups = ["sampling-strategy", "iteration", "class"]
for i in range(len(groups)):
    for g in list(itertools.permutations(groups, i+1)):
        g = list(g)
        if i == 0:
            capt = "Overview of active-learning samples grouped by "+", ".join(g)+".",
        else:
            capt = "Overview of active-learning samples grouped by "+", ".join(g[:-1])+" and "+ g[-1]+".",
        t = df_l.groupby(g).agg({"labeled": "count", "loss": "sum", "unexpected": "sum"}).rename(columns={"labeled": "count"})
        t.loss = t.loss.astype(str) + t.loss.div(t["count"]).mul(100).apply(lambda x: f""" ({x:.1f} \\%)""")
        t.unexpected = t.unexpected.astype(str) + t.unexpected.div(t["count"]).mul(100).apply(lambda x: f""" ({x:.1f} \\%)""")
        t.rename(columns={"labeled": "count"}).style.format().to_latex(
            table_path/str("_".join(g)+".tex"),
            position="H",
            caption=capt[0],
            label="tab:al_overview_"+"_".join(g),
            environment="longtable",
            sparse_index=True,
            sparse_columns=True,
            hrules=True)
        print("\input{tables/active-learning-stats/"+"_".join(g)+".tex}")


/tmp/ipykernel_6706/639203395.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_l["sampling-strategy"] = df_l.strategy.apply(lambda x: "least confidence" if x.split("_")[0] == "lc" else "$\mathcal{F}_{+}$")
/tmp/ipykernel_6706/639203395.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_l["class"] = df_l.strategy.apply(lambda x: x.split("_")[1] if x.split("_")[0] == "lc" else x.split("_")[2])
/tmp/ipykernel_6706/639203395.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

\input{tables/active-learning-stats/sampling-strategy.tex}
\input{tables/active-learning-stats/iteration.tex}
\input{tables/active-learning-stats/class.tex}
\input{tables/active-learning-stats/sampling-strategy_iteration.tex}
\input{tables/active-learning-stats/sampling-strategy_class.tex}
\input{tables/active-learning-stats/iteration_sampling-strategy.tex}
\input{tables/active-learning-stats/iteration_class.tex}
\input{tables/active-learning-stats/class_sampling-strategy.tex}
\input{tables/active-learning-stats/class_iteration.tex}
\input{tables/active-learning-stats/sampling-strategy_iteration_class.tex}
\input{tables/active-learning-stats/sampling-strategy_class_iteration.tex}
\input{tables/active-learning-stats/iteration_sampling-strategy_class.tex}
\input{tables/active-learning-stats/iteration_class_sampling-strategy.tex}


/home/andreas/Polybox/Project-Support-Material/Thesis/.venv/lib/python3.10/site-packages/pandas/core/groupby/ops.py:591: RuntimeWarning: overflow encountered in scalar add
  func(


\input{tables/active-learning-stats/class_sampling-strategy_iteration.tex}
\input{tables/active-learning-stats/class_iteration_sampling-strategy.tex}


\input{tables/active-learning-stats/sampling-strategy.tex}
\input{tables/active-learning-stats/iteration.tex}
\input{tables/active-learning-stats/class.tex}
\input{tables/active-learning-stats/sampling-strategy_iteration.tex}
\input{tables/active-learning-stats/sampling-strategy_class.tex}
\input{tables/active-learning-stats/iteration_sampling-strategy.tex}
\input{tables/active-learning-stats/iteration_class.tex}
\input{tables/active-learning-stats/class_sampling-strategy.tex}
\input{tables/active-learning-stats/class_iteration.tex}
\input{tables/active-learning-stats/sampling-strategy_iteration_class.tex}
\input{tables/active-learning-stats/sampling-strategy_class_iteration.tex}
\input{tables/active-learning-stats/iteration_sampling-strategy_class.tex}
\input{tables/active-learning-stats/iteration_class_sampling-strategy.tex}
\input{tables/active-learning-stats/class_sampling-strategy_iteration.tex}
\input{tables/active-learning-stats/class_iteration_sampling-strategy.tex}


/home/andreas/Polybox/Project-Support-Material/Thesis/.venv/lib/python3.10/site-packages/pandas/core/groupby/ops.py:591: RuntimeWarning: overflow encountered in scalar add
  func(


In [ ]:


    print(df_l.groupby(group).agg({"labeled": "count", "loss": "sum", "unexpected": "sum"}).rename(columns={"labeled": "count"}))

df_l.groupby(["sampling-strategy"]).agg({"labeled": "count", "loss": "sum", "unexpected": "sum"}).rename(columns={"labeled": "count"})

In [22]:
df_l.groupby(["iteration", "class"]).agg({"labeled": "count", "loss": "sum", "unexpected": "sum"})

labeled loss unexpected
iteration class                              
1         loss            200  113          4
          unexpected      300   54         34
2         loss            250   69          5
          unexpected      250   50         57
3         loss            150   81          4
          unexpected      350   84         79

In [23]:
df_l.groupby(["sampling", "iteration", "class"]).agg({"labeled": "count", "loss": "sum", "unexpected": "sum"})

labeled loss unexpected
sampling         iteration class                              
F+               1         loss            100   95          3
                           unexpected      100   27         26
                 2         loss             50   36          4
                           unexpected       50   13         20
                 3         loss             50   49          1
                           unexpected      100   28         46
least confidence 1         loss            100   18          1
                           unexpected      200   27          8
                 2         loss            200   33          1
                           unexpected      200   37         37
                 3         loss            100   32          3
                           unexpected      250   56         33

In [21]:
df_l.groupby(["class", "iteration", "sampling"]).agg({"labeled": "count", "loss": "sum", "unexpected": "sum"})

labeled loss unexpected
class      iteration sampling                                 
loss       1         F+                    100   95          3
                     least confidence      100   18          1
           2         F+                     50   36          4
                     least confidence      200   33          1
           3         F+                     50   49          1
                     least confidence      100   32          3
unexpected 1         F+                    100   27         26
                     least confidence      200   27          8
           2         F+                     50   13         20
                     least confidence      200   37         37
           3         F+                    100   28         46
                     least confidence      250   56         33

In [48]:
annotator = ActiveAnnotator(
    data = data,
    labels = {f"loss_AL_{iteration}": "loss", f"unexpected_AL_{iteration}": "unexpected"},
    out_file = base_path/f"data/labeling/active-learning-iteration-{iteration}.pkl",
    text_column = "text",
    batch_size = 25,
    target_label = f"unexpected_AL_{iteration}",
    target_true_id = 0
)

In [ ]:
annotator.label_batch(strategy="lc")

Unexpected quirks

exceptional != unexpected
forward looking statements

In [74]:
annotator.data.strategy.value_counts()

lc_unexpected_AL_2      250
lc_unexpected_AL_1      200
lc_loss_AL_1            200
lc_unexpected           200
hc_t_unexpected         100
lc_loss                 100
hc_t_loss               100
lc_loss_AL_2            100
hc_t_unexpected_AL_2    100
hc_t_loss_AL_1           50
hc_t_loss_AL_2           50
hc_t_unexpected_AL_1     50
Name: strategy, dtype: int64

In [75]:
annotator.get_annotations_count_per_label()

,loss,unexpected,labeled_examples
count,451,183,1500


In [ ]:
print("unexpected")
print((annotator.data.p_unexpected > 0.5).sum())
print((annotator.data.p_unexpected_AL_1 > 0.5).sum())
print((annotator.data.p_unexpected_AL_2 > 0.5).sum())
print("loss")
print((annotator.data.p_loss > 0.5).sum())
print((annotator.data.p_loss_AL_1 > 0.5).sum())
print((annotator.data.p_loss_AL_2 > 0.5).sum())

In [35]:
print(((annotator.data.p_loss > 0.5)&(annotator.data.p_unexpected > 0.5)).sum())
print(((annotator.data.p_loss_AL_1 > 0.5)&(annotator.data.p_unexpected_AL_1 > 0.5)).sum())
print(((annotator.data.p_loss_AL_2 > 0.5)&(annotator.data.p_unexpected_AL_2 > 0.5)).sum())

1294
717
301


# Initialize the labeling pickle for NEW round

In [24]:
import datasets
import pandas as pd
from scipy.special import softmax

iteration = 1

data_loss = datasets.load_from_disk(base_path/f"data/processed/loss/active-learning-iteration-{iteration}")

data_unexpected = datasets.load_from_disk(base_path/f"data/processed/unexpected/active-learning-iteration-{iteration}")
annotations = pd.read_pickle(Path(config["BASE_PATH"])/f"data/labeling/active-learning-iteration-{iteration-1}.pkl")

df = data_loss.to_pandas().merge(data_unexpected.to_pandas()[["report_id", "paragraph_nr", f"logits_unexpected_AL_{iteration}"]], on=["report_id", "paragraph_nr"])
df.drop(columns=["input_ids", "token_type_ids", "attention_mask", "text", "n_words", "filing_type"], inplace=True)
df[f"p_loss_AL_{iteration}"] = df[f"logits_loss_AL_{iteration}"].apply(lambda x: softmax(x)[0])
df[f"p_unexpected_AL_{iteration}"] = df[f"logits_unexpected_AL_{iteration}"].apply(lambda x: softmax(x)[0])

# Be careful with this line as it potentially overwrites already existing labels

In [ ]:
#df.merge(annotations, on=["report_id", "paragraph_nr"]).to_pickle(Path(config["BASE_PATH"])/f"data/labeling/active-learning-iteration-{iteration}.pkl")

# Unbiased Report-Wise Labeling

In [1]:
import sys
import datasets
import pandas as pd
import numpy as np
from dotenv import dotenv_values
from pathlib import Path

np.random.seed(19950808)

config = dotenv_values("./../../config/.env") # take environment variables from .env.
base_path = Path(config["BASE_PATH"])
sys.path.append(str(base_path/"code"))

In [2]:
df_main = pd.read_pickle(base_path/"data/preprocessed/main.pkl")
df_main.reset_index(inplace=True)
df_main.filing_type = df_main.filing_type.apply(lambda x: "10-K" if x[:4] == "10-K" else x)  # treat all SEC filings the same
df_main.year = df_main.year.astype(int)
df_main.report_id = df_main.report_id.astype(str)
df_main = df_main[df_main.year >= 1993] # Do not use historical Swiss Re Reports for training
df = pd.read_pickle(base_path/"data/preprocessed/paragraphs_clean.pkl")
df.report_id = df.report_id.astype(str)

In [3]:
df_zs = datasets.Dataset.load_from_disk(base_path/"data/preprocessed/dataset/zero-shot").to_pandas() 
df_zs["labeled"] = 1.0
df_zs["strategy"] = "zero-shot"
df_al = pd.read_pickle(base_path/"data/labeling/active-learning-iteration-2.pkl")
df_al = df_al[df_al.labeled == 1.0]
df_al["strategy"] = "active-learning"
df_labeled = pd.concat([df_zs[["report_id", "paragraph_nr", "labeled", "strategy"]], df_al[["report_id", "paragraph_nr", "labeled", "strategy"]]])
df_labeled.report_id = df_labeled.report_id.astype(str)

In [4]:
df = df.merge(df_labeled, on=["report_id", "paragraph_nr"], how="left")
df.labeled = df.labeled.apply(lambda x: x if x == 1.0 else 0.0)
df.sort_values(by=["report_id", "paragraph_nr"], inplace=True)

In [5]:
reports_to_label = df_main.groupby("filing_type").sample(5)["report_id"].tolist()

In [6]:
df.loc[df.report_id.isin(reports_to_label) & df.labeled.isin([0]), "labeled"] = -1

In [7]:
df["loss"] = None
df["unexpected"] = None
df.reset_index(drop=True, inplace=True)
df_report_id =df.report_id.astype(str)
df = df.merge(df_main.drop("filing_type", axis=1), how ="left", on="report_id")

In [8]:
df.to_pickle(base_path/"data/labeling/GT.pkl")

In [12]:
import sys
import pandas as pd
import numpy as np
from dotenv import dotenv_values
from pathlib import Path

np.random.seed(19950808)

config = dotenv_values("./../../config/.env") # take environment variables from .env.
base_path = Path(config["BASE_PATH"])
sys.path.append(str(base_path/"code"))

from ZeDiAc.Annotators import ReportAnnotator

In [5]:
df= pd.read_pickle(base_path/"data/labeling/GT.pkl")

In [9]:
df= pd.read_pickle(base_path/"data/labeling/GT.pkl")
tf = df[(df.labeled == True) & (df.strategy == "sequential") ]
tf

,report_id,paragraph_nr,text,n_words,filing_type,labeled,strategy,loss,unexpected,company,year
345824,3585,513,The corporate headquarters of the Company is l...,42,10K,True,sequential,0,0,ASSURANCEAMERICA CORP,2007.0
345825,3585,530,The Company reported net income of $.3 million...,108,10K,True,sequential,0,0,ASSURANCEAMERICA CORP,2007.0
345826,3585,531,Contributing factors towards the Company’s 200...,176,10K,True,sequential,1,0,ASSURANCEAMERICA CORP,2007.0
345827,3585,534,Gross premiums written for the year ended Dece...,238,10K,True,sequential,0,0,ASSURANCEAMERICA CORP,2007.0
345828,3585,535,Premiums written refers to the total amount of...,139,10K,True,sequential,0,0,ASSURANCEAMERICA CORP,2007.0
...,...,...,...,...,...,...,...,...,...,...,...
1150793,BeazleyPLC-AR_2017,2301,Sidecar special purpose syndicate Specialty re...,67,annual_report,True,sequential,0,0,BeazleyPLC,2017.0
1150794,BeazleyPLC-AR_2017,2305,Total shareholder return (TSR) The increase in...,25,annual_report,True,sequential,0,0,BeazleyPLC,2017.0
1150795,BeazleyPLC-AR_2017,2307,Unearned premiums reserve The portion of premi...,25,annual_report,True,sequential,0,0,BeazleyPLC,2017.0
1150796,BeazleyPLC-AR_2017,2309,If you have finished reading this report and n...,31,annual_report,True,sequential,0,0,BeazleyPLC,2017.0


In [11]:
tf.groupby(["filing_type", "report_id"]).agg({"labeled": "count", "loss": "sum", "unexpected": "sum"}).rename(columns={"labeled": "count"})

count loss unexpected
filing_type   report_id                                       
10K           3585                          39    3          0
              3935                           1    0          0
              4123                         157   29          4
              5110                         382   27         11
              5407                         184   10          0
annual_report BaloiseHoldingLtd-AR_2015    934   19          4
              BeazleyPLC-AR_2017          1069   49          2

In [5]:
tf = df[(df.labeled == True) & (df.strategy == "sequential") | (df.labeled == -1)]

In [6]:
tf[["report_id", "paragraph_nr", "text", "n_words"]].to_pickle(base_path/"data/labeling/Eval.pkl")

In [13]:
annotator = ReportAnnotator(
    data = " ",
    labels = {f"loss_GT": "loss", f"unexpected_GT": "unexpected"},
    out_file = base_path/"data/labeling/GT.pkl",
    text_column = "text",
    batch_size = 50,
)

In [15]:
annotator.data[(annotator.data.company == 'ALLSTATE LIFE INSURANCE CO') & (annotator.data.year == 2009)].labeled.value_counts()

True    367
-1      180
Name: labeled, dtype: int64

In [14]:
annotator.label_batch(strategy="sequential")

Companies:  ['FIRST COMMONWEALTH CORP'] Years:  [1999.] Paragraphs: 305 to 407


HTML(value='0 of 50 Examples annotated, Current Position: 0 ')

Output()

In [14]:
n_annotated = annotator.data[annotator.data.strategy == "sequential"].shape[0]
n_to_annotate = annotator.data[annotator.data.labeled==-1].shape[0]
n_sample = n_annotated + n_to_annotate
print(f"Annotated this session: {n_annotated-2766}")
print(f"Remaining:             {n_to_annotate} of {n_sample} ({n_to_annotate/n_sample:.1%})")
print(f"Annotated:             {n_annotated} of {n_sample} ({n_annotated/n_sample:.1%})")
counts = annotator.data[annotator.data.strategy.isin(["sequential"])][["loss", "unexpected"]].sum()
print(f"Labeled as Loss:        {counts.tolist()[0]} ({round(counts.tolist()[0]/n_annotated*100,2)}%)")
print(f"Labeled as Unexpected:   {counts.tolist()[1]} ({round(counts.tolist()[1]/n_annotated*100,2)}%)")

Annotated this session: 1250
Remaining:             2056 of 6072 (33.9%)
Annotated:             4016 of 6072 (66.1%)
Labeled as Loss:        228 (5.68%)
Labeled as Unexpected:   26 (0.65%)


In [18]:
annotator.data[annotator.data.labeled.isin([-1]) & annotator.data.company.isin(['BeazleyPLC'])].shape[0]

50

In [30]:
df_main = pd.read_pickle(base_path/"data/preprocessed/main.pkl")
df_main.reset_index(inplace=True)
df_main.filing_type = df_main.filing_type.apply(lambda x: "10-K" if x[:4] == "10-K" else x)  # treat all SEC filings the same
df_main.year = df_main.year.astype(int)
df_main.report_id = df_main.report_id.astype(str)
df_main = df_main[df_main.year >= 1993]
df_main.filing_type.value_counts()

10-K             5959
annual_report     616
Name: filing_type, dtype: int64

In [31]:
5959/616

9.6737012987013

In [32]:
df = pd.read_pickle(base_path/"data/labeling/GT.pkl")

In [34]:
df[df.strategy == "sequential"].groupby("report_id").filing_type.value_counts()

report_id                  filing_type  
3585                       10K                39
3935                       10K                 1
4123                       10K               157
5110                       10K               382
5407                       10K               184
BaloiseHoldingLtd-AR_2015  annual_report     934
BeazleyPLC-AR_2017         annual_report    1069
Name: filing_type, dtype: int64

In [36]:
2*5959/616 - 5

14.3474025974026

In [45]:
labeled_10_k_s = df[(df.strategy == "sequential")& (df.filing_type == "10K")].report_id.unique()

In [57]:
np.random.seed(19950808)
reports_to_label = df_main[(~df_main.report_id.isin(labeled_10_k_s)) & df_main.filing_type.isin(["10-K"])].sample(14).report_id.tolist()

In [60]:
df.loc[df.report_id.isin(reports_to_label) & df.labeled.isin([0]), "labeled"] = -1
df.to_pickle(base_path/"data/labeling/GT.pkl")